## 1. Get all questions

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from text_untils import *
import json

In [2]:
def getCollection(database, collection):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database]
    cl = db[collection]

    pipeline = [
        {
            "$match": {
                "title": {"$nin": [None, ""]},
            }
        }
    ]
    docs = list(cl.aggregate(pipeline))
    # client.close()
    return docs

In [3]:
documents = getCollection('lawlaboratory', 'temp')

In [4]:
df = pd.DataFrame(documents)
df.sample(5)

,_id,title,description,date_answer,field,source_url,conclusion,reference,quote
15940,666342eae485130f03f513b4,Quyền vận tải biển nội địa theo Luật Hàng hải ...,Hoạt động hàng hải là một trong những ngành qu...,2018-07-25,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/3...,[Quyền vận tải biển nội địa được quy định tại ...,NaN,NaN
15784,666342ede485130f03f54ca5,Quyền bảo vệ của các quốc gia ven biển được qu...,Quyền bảo vệ của các quốc gia ven biển được qu...,2018-05-24,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/3...,[Căn cứ theo Điều 25 Công ước về Luật biển năm...,NaN,NaN
26793,666342eee485130f03f5579b,Đơn giản hóa thủ tục kéo dài thời hạn tạm giữ ...,Đơn giản hóa thủ tục kéo dài thời hạn tạm giữ ...,2017-09-27,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/3...,[Đơn giản hóa thủ tục kéo dài thời hạn tạm giữ...,NaN,NaN
3199,666342ede485130f03f54333,Các hạng mục công trình tại trạm dừng nghỉ đườ...,Các hạng mục công trình tại trạm dừng nghỉ đườ...,2018-08-10,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/3...,[Các hạng mục công trình tại trạm dừng nghỉ đư...,NaN,NaN
8189,666342eee485130f03f55715,Khai báo tai nạn lao động hàng hải như thế nào?,"Chào Ban biên tập, tôi là Thanh Minh, hiện tôi...",2019-02-20,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,[Việc khai báo tai nạn lao động hàng hải được ...,NaN,NaN


In [5]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27716 entries, 0 to 27715
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          27716 non-null  object
 1   title        27716 non-null  object
 2   description  20796 non-null  object
 3   date_answer  27716 non-null  object
 4   field        27716 non-null  object
 5   source_url   27716 non-null  object
 6   conclusion   27716 non-null  object
 7   reference    6920 non-null   object
 8   quote        6920 non-null   object
dtypes: object(9)
memory usage: 1.9+ MB


## 2. Data cleaning

In [6]:
df = df.dropna(subset=['title'])
df = df[df['title'] != '']

In [7]:
def dropEmpty(row):
    if isinstance(row['conclusion'], list) and not row['conclusion'] and pd.isna(row['quote']):
        return True
    return False

df = df[~df.apply(dropEmpty, axis=1)]

df = df.reset_index(drop=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27411 entries, 0 to 27410
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          27411 non-null  object
 1   title        27411 non-null  object
 2   description  20491 non-null  object
 3   date_answer  27411 non-null  object
 4   field        27411 non-null  object
 5   source_url   27411 non-null  object
 6   conclusion   27411 non-null  object
 7   reference    6920 non-null   object
 8   quote        6920 non-null   object
dtypes: object(9)
memory usage: 1.9+ MB


In [9]:
viet_characters = "ÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬàáảãạăằắẳẵặâầấẩẫậĐđÈÉẺẼẸÊỀẾỂỄỆèéẻẽẹêềếểễệÌÍỈĨỊìíỉĩịÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢòóỏõọôồốổỗộơờớởỡợÙÚỦŨỤƯỪỨỬỮỰùúủũụưừứửữựỲÝỶỸỴỳýỷỹỵ"

def contains_vietnamese_characters(title, vietnamese_chars):
    return any(char in vietnamese_chars for char in title)

df = df[df['title'].apply(lambda x: contains_vietnamese_characters(x, viet_characters))]

In [10]:
def process_quote(quote):
    if pd.isna(quote):
        return np.nan
    if 'content' in quote and not quote['content']:
        return np.nan
    return quote

df['quote'] = df['quote'].apply(process_quote)

In [11]:
def dropEmptyAnswer(row):
    if isinstance(row['conclusion'], list) and not row['conclusion'] and pd.isna(row['quote']):
        return True
    return False

df = df[~df.apply(dropEmptyAnswer, axis=1)]

df = df.reset_index(drop=True)
df.shape

(27279, 9)

In [12]:
def clean_conclusionArray(conclusion):
    words_to_remove = ["tải về", "mẫu đơn", "hình từ internet", "hình ảnh internet", "hình internet", "trân trọng"]
    if isinstance(conclusion, list):
        cleaned_conclusion1 = [word for word in conclusion if isinstance(word, str) and not any(remove_word in word.lower() for remove_word in words_to_remove)]
        cleaned_conclusion2 = [x for x in cleaned_conclusion1 if x]
        return cleaned_conclusion2
    return conclusion

df['conclusion'] = df['conclusion'].apply(clean_conclusionArray)

In [13]:
def clean_title(title):
    for i, char in enumerate(title):
        if char.isupper():
            return title[i:]
    return title

df['title'] = df['title'].apply(clean_title)

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [15]:
df['token_count'] = df['title'].apply(count_tokens)
df = df[df['token_count'] > 3].drop(columns=['token_count'])

In [16]:
df['date_answer'] = pd.to_datetime(df['date_answer'])

df.sort_values(by='date_answer', ascending=False, inplace=True)
df.drop_duplicates(subset='title', keep='first', inplace=True)

df['date_answer'] = df['date_answer'].dt.strftime('%Y-%m-%d')

In [17]:
df.reset_index(drop=True, inplace=True)
df.shape

(27262, 9)

In [18]:
df.to_json('data/backup/temp.json', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [19]:
def merge_quote(quote):
    if isinstance(quote, dict):
        name = quote.get('name', ' ')
        content = '\n'.join(quote.get('content', []))

        if name is None:
            name = ' '
        if content is None:
            content = ' '

        return f"{name}: \n {content}"
    else:
        return None


df['conclusion'] = df.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)

In [20]:
df = df.drop(['date_answer', 'field', 'description', 'source_url'], axis=1)

In [21]:
df.sample(5)

,_id,title,conclusion,reference,quote
4976,666342ece485130f03f54025,Quy chuẩn tính năng chấm điểm và thông báo lỗi...,: \n Với bài tập sa hình PMGV phải cho phép c...,Tại Điểm c Tiểu tiết 2.2.3.4 Tiết 2.2.3 Tiểu m...,{'content': ['Với bài tập sa hình PMGV phải ch...
12282,666342ebe485130f03f522b5,CSGT có quyền yêu cầu dừng xe đang lưu thông đ...,[Căn cứ theo quy định tại Khoản 2 Điều 12 Thôn...,NaN,NaN
9156,666342eae485130f03f50fc1,Bản vẽ kỹ thuật của gương dùng cho xe ô tô phả...,[Căn cứ tiết 3.2.1.1 Mục 3 Quy chuẩn kỹ thuật ...,NaN,NaN
25684,666342ede485130f03f5495b,Ô tô khi lưu thông trong hầm đường bộ không bậ...,[Các phương tiện lưu thông trong hầm đường bộ ...,NaN,NaN
5261,666342efe485130f03f56ba2,"Phối hợp trong công tác đào tạo, huấn luyện ng...",[Căn cứ Điều 7 Thông tư liên tịch 63/2016/TTLT...,Căn cứ Điều 7 Thông tư liên tịch 63/2016/TTLT-...,NaN


In [22]:
def printConclustion(df, target_title):
    conclusion_value = df.loc[df['title'] == target_title, 'conclusion'].values[0]

    if isinstance(conclusion_value, list):
        conclusion_value = '\n'.join(conclusion_value)

    if conclusion_value and isinstance(conclusion_value, str):
        print(f"'{target_title}': {conclusion_value}")
    else:
        print(f"'{target_title}'")

printConclustion(df, "Trường hợp được xem là vi phạm thời gian lái xe khi tham gia giao thông?")

'Trường hợp được xem là vi phạm thời gian lái xe khi tham gia giao thông?': Như vậy, vi phạm thời gian lái xe khi tham gia giao thông được xác định như sau:
- Vi phạm thời gian lái xe liên tục được xác định như sau:
+ Khi có thời gian lái xe của một người vượt quá 4 giờ nhưng không dừng, đỗ xe >= 15 phút (áp dụng đối với xe tuyến cố định, xe hợp đồng, xe du lịch, xe buýt liên tỉnh và xe vận tải hàng hoá); >= 5 phút (áp dụng đối với xe buýt nội tỉnh, xe taxi) hoặc không đổi lái xe.
+ Đối với các đơn vị có truyền thông tin bật/tắt động cơ xe là một điều kiện tính toán, trong trường hợp phương tiện vẫn di chuyển nhưng không có dữ liệu thì vẫn tính thời gian lái xe liên tục.
- Vi phạm thời gian lái xe trong ngày: được xác định khi tổng thời gian lái xe của một người trong ngày vượt quá 10 giờ.


In [23]:
def create_answer(row):
    conclusion_value = row['conclusion']

    if isinstance(conclusion_value, list):
        conclusion_value = ' '.join(conclusion_value)
    elif pd.isna(conclusion_value):
        conclusion_value = ''

    quote_value= ''
    if not pd.isna(row['quote']):
        reference = row['reference'] if not pd.isna(row['reference']) else ''
        merged_quote = merge_quote(row['quote'])
        quote_value = f"{reference} {merged_quote}"

    conclusion_tokens = count_tokens(conclusion_value)
    quote_tokens = count_tokens(quote_value)

    if conclusion_tokens >= quote_tokens:
        return conclusion_value
    else:
        return quote_value

df['answer'] = df.apply(create_answer, axis=1)

In [24]:
df['title'] = df['title'].apply(clean_text)
df['answer'] = df['answer'].apply(clean_text)

In [25]:
df['title'] = df['title'].apply(word_segment)
df['answer'] = df['answer'].apply(word_segment)

In [26]:
df['title'] = df['title'].apply(normalize_text)
df['answer'] = df['answer'].apply(normalize_text)

In [27]:
df['title'] = df['title'].apply(remove_stopword)
df['answer'] = df['answer'].apply(remove_stopword)

In [28]:
# df['title'] = df['title'].apply(strip_answer_string)
# df['conclusion'] = df['conclusion'].apply(strip_answer_string)

In [29]:
df['title'] = df['title'].str.replace('_', ' ')
df['answer'] = df['answer'].str.replace('_', ' ')

In [30]:
df = df.drop_duplicates(subset='title', keep = 'first')
df.duplicated(subset='title').sum()

0

In [31]:
df.to_json('data/cleaned/temp.json', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)